In [1]:
# #!pip install patchify
# !pip3 install segmentation_models
# !pip install -U -q segmentation-models
# !pip install -q tensorflow==2.2.1
# !pip install -q keras==2.5

In [2]:
from google  import colab
from  google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/path/to/tensorflow/keras/metrics')
!pip install segmentation_models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.2 MB/s eta 0:00:00


In [4]:
!pip install --upgrade segmentation_models
!pip install keras==2.6.0
!pip install patchify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.6.0 which is incompatible.


In [5]:
!pip install keras==2.6.0

In [6]:
!pip install --upgrade keras segmentation_models



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


In [7]:
!pip install tensorflow
!pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.0.5
    Uninstalling keras-3.0.5:
      Successfully uninstalled keras-3.0.5


In [8]:
!pip install --upgrade keras

  Using cached keras-3.0.5-py3-none-any.whl (1.0 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


In [12]:
!pip install --upgrade keras
!pip install --upgrade segmentation_models

import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from  patchify import patchify
from PIL import Image
#import segmentation_models as sm
from tensorflow import keras
from  tensorflow.keras.metrics import MeanIoU
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from sklearn.model_selection import train_test_split
import sys
sys.path.append('/content/drive/MyDrive/AerialImageryDataset')
import simple_multi_unet_model
from simple_multi_unet_model import multi_unet_model , jacard_coef

In [13]:
scaler = MinMaxScaler()

In [14]:
root_dir = '/content/drive/MyDrive/AerialImageryDataset'

In [15]:
patch_size = 256

In [16]:
image_dataset = []
for path , sub_dir  , files in os.walk(root_dir):
  dirname = path.split(os.path.sep)[-1]
  if dirname == 'images': #find files with name "images" or "image" directory
    images = os.listdir(path) #list of all image names in this subdirectory
    for i , image_name in enumerate(images):
      if image_name.endswith(".jpg"):#read only jpeg images
        image = cv.imread(path + '/' + image_name , 1) # 1 for reading rgb image #reading the path names
        size_x = (image.shape[1]//patch_size)*patch_size #scaling factors of x and y
        size_y = (image.shape[0]//patch_size)*patch_size
        image = Image.fromarray(image)#forming array of the image
        image = image.crop((0,0,size_x , size_y))

        image = np.array(image)#forming numpy array of images
        #extracting  patches
        print("now patchifying image " , path +"/" + image_name )
        patches_img =patchify(image , (patch_size , patch_size , 3) , step = patch_size) #no overlapping
        for i in  range(patches_img.shape[0]):
          for j in range(patches_img.shape[1]):
            single_patch_img = patches_img[i , j ,: , :]
            single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape)

            single_patch_img = single_patch_img[0]
            image_dataset.append(single_patch_img)



now patchifying image  /content/drive/MyDrive/AerialImageryDataset/Tile 6/images/image_part_009.jpg
now patchifying image  /content/drive/MyDrive/AerialImageryDataset/Tile 6/images/image_part_005.jpg
now patchifying image  /content/drive/MyDrive/AerialImageryDataset/Tile 6/images/image_part_001.jpg
now patchifying image  /content/drive/MyDrive/AerialImageryDataset/Tile 6/images/image_part_002.jpg
now patchifying image  /content/drive/MyDrive/AerialImageryDataset/Tile 6/images/image_part_004.jpg
now patchifying image  /content/drive/MyDrive/AerialImageryDataset/Tile 6/images/image_part_006.jpg
now patchifying image  /content/drive/MyDrive/AerialImageryDataset/Tile 6/images/image_part_007.jpg
now patchifying image  /content/drive/MyDrive/AerialImageryDataset/Tile 6/images/image_part_003.jpg
now patchifying image  /content/drive/MyDrive/AerialImageryDataset/Tile 6/images/image_part_008.jpg
now patchifying image  /content/drive/MyDrive/AerialImageryDataset/Tile 5/images/image_part_007.jpg


In [17]:
#doing the same with masks
mask_dataset = []
for path , subdirs  , files in os.walk(root_dir):
  dirname = path.split(os.path.sep)[-1]
  if dirname == 'masks': #find files with name "images" or "image" directory
    masks = os.listdir(path) #list of all image names in this subdirectory
    for i ,mask_name in enumerate(masks):
      if mask_name.endswith(".png"):#read only jpeg images
        mask = cv.imread(path + '/' + mask_name , 1) # 1 for reading rgb image #reading the path names
        mask = cv.cvtColor(mask , cv.COLOR_BGR2RGB)
        size_x = (mask.shape[1]//patch_size)*patch_size #scaling factors of x and y
        size_y = (mask.shape[0]//patch_size)*patch_size
        mask =Image.fromarray(mask)#forming array of the image
        mask =mask.crop((0,0,size_x , size_y))

        mask = np.array(mask )#forming numpy array of images
        #extracting  patches
        print("now patchifying mask " , path +"/" + mask_name )
        patches_mask =patchify(mask , (patch_size , patch_size , 3) , step = patch_size) #no overlapping
        for i in  range(patches_mask.shape[0]):
          for j in range(patches_mask.shape[1]):
            single_patch_mask = patches_mask[i , j ,: , :]
            single_patch_mask = single_patch_mask[0]
            mask_dataset.append(single_patch_mask)


now patchifying mask  /content/drive/MyDrive/AerialImageryDataset/Tile 6/masks/image_part_004.png
now patchifying mask  /content/drive/MyDrive/AerialImageryDataset/Tile 6/masks/image_part_007.png
now patchifying mask  /content/drive/MyDrive/AerialImageryDataset/Tile 6/masks/image_part_008.png
now patchifying mask  /content/drive/MyDrive/AerialImageryDataset/Tile 6/masks/image_part_003.png
now patchifying mask  /content/drive/MyDrive/AerialImageryDataset/Tile 6/masks/image_part_006.png
now patchifying mask  /content/drive/MyDrive/AerialImageryDataset/Tile 6/masks/image_part_001.png
now patchifying mask  /content/drive/MyDrive/AerialImageryDataset/Tile 6/masks/image_part_005.png
now patchifying mask  /content/drive/MyDrive/AerialImageryDataset/Tile 6/masks/image_part_002.png
now patchifying mask  /content/drive/MyDrive/AerialImageryDataset/Tile 6/masks/image_part_009.png
now patchifying mask  /content/drive/MyDrive/AerialImageryDataset/Tile 5/masks/image_part_003.png
now patchifying mask

In [18]:
image_dataset = np.array(image_dataset)
mask_dataset = np.array(mask_dataset)

In [19]:
#print(len(image_dataset))

In [20]:
# import random
# import numpy as np
# image_number = random.randint(0 , len(image_dataset))
# plt.figure(figsize = (12,6))
# plt.subplot(121)
# plt.imshow(np.reshape(image_dataset[image_number],(patch_size , patch_size , 3)))
# plt.subplot(122)
# plt.imshow(np.reshape(mask_dataset[image_number],(patch_size , patch_size , 3)))
# plt.show()


In [21]:
#tackling mask : RGB TO HEX:HEX  is hexadecimal format
# The dataset consists of aerial imagery of Dubai obtained by MBRSC satellites and annotated with pixel-wise semantic segmentation in 6 classes. The total volume of the dataset is 72 images grouped into 6 larger tiles. The classes are:

# Building: #3C1098
# Land (unpaved area): #8429F6
# Road: #6EC1E4
# Vegetation: #FEDD3A
# Water: #E2A929
# Unlabeled: #9B9B9B

# Calculating RGB from HEX: #3C1098
# 3C = 3*16 + 12 = 60
# 10 = 1*16 + 0 = 16
# 98 = 9*16 + 8 = 152

#converting  each class's hex code to RGN

Building = '#3C1098'.lstrip('#')
Building  = np.array(tuple(int(Building[i:i+2],16)for i in  (0,2,4)))

Land = '#8429F6'.lstrip('#')
Land  = np.array(tuple(int(Land[i:i+2],16)for i in  (0,2,4)))

Road = '#6EC1E4'.lstrip('#')
Road  = np.array(tuple(int(Road[i:i+2],16)for i in  (0,2,4)))

Vegetation = '#FEDD3A'.lstrip('#')
Vegetation  = np.array(tuple(int(Vegetation[i:i+2],16)for i in  (0,2,4)))

Water = '#E2A929'.lstrip('#')
Water  = np.array(tuple(int(Water[i:i+2],16)for i in  (0,2,4)))

Unlabeled = '#9B9B9B'.lstrip('#')
Unlabeled  = np.array(tuple(int(Unlabeled[i:i+2],16)for i in  (0,2,4)))

label = single_patch_mask

In [22]:
print(Building)

[ 60  16 152]


In [23]:
#replacing rgb to intefer then to one hot encoded
#for this replacing rgb to integers values to be used as labels , find pixels with combination of RGB for above defined arrays , if matched then replace all values in that pixels with a specific integer
def rgb_2_2Dlabels(label):
  label_seg = np.zeros(label.shape , dtype = np.uint8)
  label_seg[np.all(label == Building , axis = -1)] = 0
  label_seg[np.all(label ==Land , axis = -1)] = 1
  label_seg[np.all(label == Road , axis = -1)] = 2
  label_seg[np.all(label == Vegetation , axis = -1)] = 3
  label_seg[np.all(label == Water , axis = -1)] = 4
  label_seg[np.all(label == Unlabeled , axis = -1)] = 5

  label_seg = label_seg[:,:,0]
  return label_seg






In [24]:
labels = []
for i in range(mask_dataset.shape[0]):
  label = rgb_2_2Dlabels(mask_dataset[i])
  labels.append(label)


In [25]:
labels = np.array(labels)
labels = np.expand_dims(labels , axis = 3)

In [26]:
print("unique labels : " , np.unique(labels))


unique labels :  [0 1 2 3 4 5]


In [27]:
# import random
# import numpy as np
# image_number = random.randint(0 , len(image_dataset))
# plt.figure(figsize = (12,6))
# plt.subplot(121)
# plt.imshow(image_dataset[image_number])
# plt.subplot(122)
# plt.imshow(labels[image_number][:,:,0])
# plt.show()

In [28]:
#one hot encoding
n_classes = len(np.unique(labels))
from tensorflow.keras.utils import to_categorical

labels_cat = to_categorical(labels , num_classes = n_classes)

In [29]:
print(labels_cat.shape)

(1325, 256, 256, 6)


In [ ]:


# Check the number of samples in each dataset
num_samples_image = len(image_dataset)
num_samples_labels = len(labels_cat)

# Ensure both datasets have the same number of samples
if num_samples_image != num_samples_labels:
    min_samples = min(num_samples_image, num_samples_labels)
    image_dataset = image_dataset[:min_samples]
    labels_cat = labels_cat[:min_samples]
    print("Number of samples adjusted to:", min_samples)

# Split the datasets into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(image_dataset, labels_cat, test_size=0.2, random_state=42)


Number of samples adjusted to: 1305


In [ ]:
print(x_train.shape)

In [ ]:
weights = [0.1666,0.1666,0.1666,0.1666,0.1666,0.1666]
dice_loss = sm.losses.DiceLoss(class_weights = weights)
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1*focal_loss)

img_height = x_train.shape[1]
img_width = x_train.shape[2]
img_channels = x_train.shape[3]


In [ ]:

metrics = ['accuracy',jacard_coef]
def get_model():
  return multi_unet_model(n_classes = n_classes , IMG_HEIGHT = img_height , IMG_WIDTH = img_width , IMG_CHANNELS = img_channels)



In [ ]:
model = get_model()
model.compile(loss = total_loss , optimizer = 'adam' , metrics = metrics)
model.summary

In [ ]:
fitting = model.fit(x_train , y_train , batch_size = 16, verbose = 1 , epochs  = 10 , validation_data = (x_test , y_test ) , shuffle = False)